In [36]:
from app.template.variable.expr import LazyExpr
from typing import List, Dict, Any, Optional
from datetime import datetime
import ast

In [37]:
im = ["import numpy as np", "from datetime import datetime"]

# s = "lambda x: all(isinstance(i, datetime) for i in x) if isinstance(x, list) else False"
s = "(aaaaa := a + 1)"
# s = """
# 1+1
# """
f = LazyExpr(s, im)

# print(ast.dump(f._ast, indent=2))
print(f.unbound)
print(f.target)

f.eval(env={"a": 1024})

{'a'}
aaaaa


1025

In [38]:
from app.template.data_source.model import DictSource

s = DictSource(name="test",
               data={
                   "a": 1,
                   "b": 2,
                   "c": 3
               },
               json_schema={
                   "type": "object",
                   "properties": {
                       "a": {
                           "type": "integer"
                       },
                       "b": {
                           "type": "integer"
                       },
                       "c": {
                           "type": "integer"
                       }
                   }
               })
s.load()

Ok({'a': 1, 'b': 2, 'c': 3})

In [39]:
from app.template.variable.model import IVariable, LiteralVariable

LiteralVariable(name="s", expr=LazyExpr("1+1"))

LiteralVariable(name='s', expr=LazyExpr(raw='1+1', imports=[]), comment=None, formatter=None, t=None)

In [40]:
var_a = LiteralVariable.model_validate({
    "name": "a",
    "expr": "datetime.now()",
    "t": "datetime",
    "imports": ["from datetime import datetime"],
})

var_a.load()

Ok(datetime.datetime(2024, 3, 28, 2, 15, 46, 743951))

In [41]:
# var_s = LiteralVariable(name="s", expr=LazyExpr("datetime.now()"), t=datetime, imports=im)
var_s = LiteralVariable(name="s",
                        expr="datetime.now()",
                        formatter="lambda x: x.strftime('%Y-%m-%d %H:%M:%S')",
                        imports=im,
                        t=datetime)
identity = lambda x: x
display(var_s.model_dump())
var_s_val = var_s.load().unwrap()
format_fn = var_s.eval_formatter()
assert format_fn is not None
display(format_fn(var_s_val))

{'name': 's',
 'expr': {'raw': 'datetime.now()',
  'imports': ['import numpy as np', 'from datetime import datetime']},
 'comment': None,
 'formatter': {'raw': "lambda x: x.strftime('%Y-%m-%d %H:%M:%S')",
  'imports': ['import numpy as np', 'from datetime import datetime']},
 't': datetime.datetime}

'2024-03-28 02:15:46'

In [42]:
lst = [
    LiteralVariable(name="s", expr=LazyExpr("1+1"), formatter="lambda x: x+1"),
    LiteralVariable(name="a", expr=LazyExpr("1+1")),
    LiteralVariable(name="b", expr=LazyExpr("a+1")),
    LiteralVariable(name="c", expr=LazyExpr("b+1")),
    LiteralVariable(name="d", expr=LazyExpr("a+b+2")),
    LiteralVariable(name="e", expr=LazyExpr("c+d+3")),
]
for v in lst:
    print(v.unbound)

set()
set()
{'a'}
{'b'}
{'a', 'b'}
{'d', 'c'}


In [43]:
from app.template.dependency.resolver import resolve_and_evaluate, to_env_dict

vars = resolve_and_evaluate(lst)
display(vars)
display(to_env_dict(vars))

[EvaluatedVariable(name='a', value=2, formatter=None),
 EvaluatedVariable(name='b', value=3, formatter=None),
 EvaluatedVariable(name='c', value=4, formatter=None),
 EvaluatedVariable(name='d', value=7, formatter=None),
 EvaluatedVariable(name='e', value=14, formatter=None),
 EvaluatedVariable(name='s', value=2, formatter=<function __lazy_expr.<locals>.<lambda> at 0x000002A647EAF7E0>)]

{'a': 2, 'b': 3, 'c': 4, 'd': 7, 'e': 14, 's': 2}

In [44]:
text = "本期中出院病例${{'a':0}}例。手术人数${num_operation}人，四级手术${num_operation_4}人，微创手术${num_operation_micro}人。CMI值${CMI}，其中 CMI 小于1的病人数有${query(CMI<1)}人，占比${query(CMI<1)/num_patient}；1-2的病人数${query(CMI > 1 and CMI < 2)}人，占比${query(CMI > 1 and CMI < 2) / num_patient};2-5的病人数${(num_CIM_2_5 := query(CMI > 2 and CMI < 5))}人，占比${num_CIM_2_5 / num_patient};大于5的病人数${(num_CIM_gt_5 := query(CMI > 5))}人,占比${num_CIM_gt_5 / num_patient}。总体来说大于2的病人数${(num_CIM_gt_2 := query(CMI > 2))}人，占比${num_CIM_gt_2 / num_patient}%，有提升空间。"

In [45]:
from app.template.content import HtmlContent, TableContent

parsed = HtmlContent(tag="p", content=text)

e = parsed.extract(None).unwrap()
display(e.text_with_hash)
display(e.exprs)
# display(list(map(lambda x: x[1].solely_dependency, e.exprs.items())))

'本期中出院病例#938344a3#例。手术人数#e2264901#人，四级手术#d5191cf5#人，微创手术#272aea71#人。CMI值#50c182b7#，其中 CMI 小于1的病人数有#44224433#人，占比#9b57cb36#；1-2的病人数#0bc37be4#人，占比#0c46628a#;2-5的病人数#80ec94a6#人，占比#f6d11d05#;大于5的病人数#0919f947#人,占比#682db670#。总体来说大于2的病人数#023ed6d7#人，占比#c33640e8#%，有提升空间。'

{'938344a3': LazyExpr(raw="{'a':0}", imports=None),
 'e2264901': LazyExpr(raw='num_operation', imports=None),
 'd5191cf5': LazyExpr(raw='num_operation_4', imports=None),
 '272aea71': LazyExpr(raw='num_operation_micro', imports=None),
 '50c182b7': LazyExpr(raw='CMI', imports=None),
 '44224433': LazyExpr(raw='query(CMI<1)', imports=None),
 '9b57cb36': LazyExpr(raw='query(CMI<1)/num_patient', imports=None),
 '0bc37be4': LazyExpr(raw='query(CMI > 1 and CMI < 2)', imports=None),
 '0c46628a': LazyExpr(raw='query(CMI > 1 and CMI < 2) / num_patient', imports=None),
 '80ec94a6': LazyExpr(raw='(num_CIM_2_5 := query(CMI > 2 and CMI < 5))', imports=None),
 'f6d11d05': LazyExpr(raw='num_CIM_2_5 / num_patient', imports=None),
 '0919f947': LazyExpr(raw='(num_CIM_gt_5 := query(CMI > 5))', imports=None),
 '682db670': LazyExpr(raw='num_CIM_gt_5 / num_patient', imports=None),
 '023ed6d7': LazyExpr(raw='(num_CIM_gt_2 := query(CMI > 2))', imports=None),
 'c33640e8': LazyExpr(raw='num_CIM_gt_2 / num_patient

In [46]:
ll = TableContent(data={
    "a": [1, 2, 3],
    "b": "${v}",
    "c": "${v} ${vv} ${sss}", # error
})

ll.extract(None)

Err(ValueError('Failed to parse some expressions', {'c': ValueError('Invalid interpolation for c `${v} ${vv} ${sss}`. Expected exactly one expression, got 3')}))

In [47]:
import orjson
from app.template import evaluate_template


tt = {
    "imports": ["import numpy as np", "from datetime import datetime", "import builtins"],
    "data_sources": [
        {
            "source_type": "dict",
            "name": "test",
            "data": {"a": 1, "b": 2, "c": 3},
            "json_schema": {
                "type": "object",
                "properties": {
                    "a": {"type": "integer"},
                    "b": {"type": "integer"},
                    "c": {"type": "integer"},
                },
            },
        }
    ],
    "variables": [
        {
            "name": "s",
            "expr": "datetime.now()",
            "formatter": "lambda x: x.strftime('%Y 年 %m 月 %d 日 %H:%M:%S')",
            "t": "datetime",
        },
        {
            "name": "a",
            "source": "test",
            "json_path": "$.a",
            "t": "int",
        },
        {
            "name": "b",
            "source": "test",
            "json_path": "$.b",
            "preprocessor": "lambda x: builtins.str(x) + '1'",
            "t": "str",
        },
        {
            "name": "c",
            "source": "test",
            "json_path": "$.c",
        },
    ],
    "contents": [
        {"tag": "p", "content": "时间是 ${s}; a 是 ${a}, b 是 ${b}, c 是 ${c}。"},
        {
            "table_type": "table",
            "data": {
                "col_1": "${[a, b, c]}",
            },
        },
    ],
}

d = await evaluate_template(tt)
display(orjson.dumps(d).decode())

'{"variables":[{"c":3},{"b":"21"},{"a":1},{"s":"2024-03-28T02:15:47.023940"}],"contents":[{"tag":"p","text":"时间是 2024 年 03 月 28 日 02:15:47; a 是 1, b 是 21, c 是 3。","style":{}},{"data":{"col_1":[1,"21",3]},"table_type":"table"}]}'